In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_mldata
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

#import warnings
#warnings.filterwarnings('ignore')

mnist = fetch_mldata('MNIST original')
X, y = mnist["data"], mnist["target"]

X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
shuffle_index = np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
#scaling the dataset for better accuracy before running the classifiers
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))


from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict


# #using randomforestclassifier
# forest_clf = RandomForestClassifier(random_state = 42)
# frst_scrs = cross_val_score(forest_clf, X_train_aug, y_train_aug, cv = 3, scoring = "accuracy", n_jobs = -1)
# print("(Random Forest)", frst_scrs)


# knn_clf = KNeighborsClassifier()
# knn_scrs = cross_val_score(knn_clf, X_train_aug, y_train_aug, cv = 3, scoring = "accuracy", n_jobs = -1)
# print("(knn classifier)", knn_scrs)

# y_train_knn_pred = cross_val_predict(knn_clf, X_train, y_multilabel, cv=3, n_jobs=-1)
# f1_score(y_multilabel, y_train_knn_pred, average="macro")


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

params =[
   # {'n_neighbors': [ 9, 13] },
    { 'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    
]

#newgridsearch = RandomizedSearchCV(forest_clf, param_distributions = params, n_iter=10, cv = 5, scoring='accuracy')
# newgridsearch = GridSearchCV(knn_clf, param_grid = params, cv = 5, scoring = "accuracy", n_jobs = -1)
newgridsearch = GridSearchCV(forest_clf, param_grid = params, cv = 5, scoring = "accuracy", n_jobs = -1)
r_grid_search = newgridsearch.fit(X_train_scaled, y_train)
r_cv = r_grid_search.cv_results_


for mean_score, params in zip(r_cv["mean_test_score"], r_cv["params"]):
    print((mean_score), params)
    
print(r_grid_search.best_estimator_)

In [3]:

#shifting the image in 4 directions. 5 pixels
from scipy.ndimage import interpolation

X_train_combined = X_train_scaled.copy().tolist()

for i in ([[-2,0], [2,0], [0,2], [0,-2]]):   
    for image in X_train_scaled:
        X_train_combined.append(interpolation.shift(image.reshape(28,28), i).reshape(-1))  

X_train_aug = np.array(X_train_combined)
print("training data shape ", X_train_aug.shape)



#making y_aug
y_train_list = y_train.tolist()
y_train_repeated = y_train_list * 5
y_train_aug = np.array(y_train_repeated)
print("training labels shape ", y_train_aug.shape)



#randomizing
shuffle_index = np.random.permutation(300000)
X_train_aug, y_train_aug = X_train_aug[shuffle_index], y_train_aug[shuffle_index]

training data shape  (300000, 784)
training labels shape  (300000,)


In [ ]:
fin_model = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=20, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

pl = cross_val_score(fin_model, X_train_aug, y_train_aug, cv = 3, scoring = "accuracy", n_jobs = -1)
print(pl)